<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/main/ADP_draft_simulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Block 1 - Setup & Datenquellen

In [ ]:
import pandas as pd
import numpy as np
import re
import random
from math import ceil
import time
import json
!pip install rapidfuzz
from rapidfuzz import process

In [ ]:
# Konfiguration
year = 2024
num_teams = 12
num_rounds = 15
num_weeks = 17
allowed_positions = {'QB', 'RB', 'WR', 'TE', 'K', 'DST'}
lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'FLEX': 1, 'K': 1, 'DST': 1}

# ADP und Projections laden
adp_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/pre_season_data/adp_projections_{year}.csv'
player_adp = (
    pd.read_csv(adp_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'adp': 'ADP', 'points': 'TTL'})
      .fillna({'ADP': 999})
      .loc[:, ['Player', 'POS', 'ADP', 'TTL']]
      .sort_values(by = ['ADP', 'TTL'], ascending = [True, False])
      .reset_index(drop=True)
      .drop_duplicates(subset='Player', keep='first')
)
player_adp['Rank'] = player_adp.index + 1

# Real Data (Punkte aus echten Spielen) laden
real_data_RZ_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/Weekly_Data/weekly_data_{year}.csv'
real_data_RZ = (
    pd.read_csv(real_data_RZ_url)
    .rename(columns={'position': 'POS', 'player_display_name': 'Player'})
    .replace({'FB': 'RB'})
    .drop(columns = ['season'], errors='ignore')
    .loc[lambda real_data_RZ: real_data_RZ['POS'].isin(allowed_positions)]
)

real_data_FP_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/Weekly_Data/FantasyPros_Fantasy_Football_Points_PPR_{year}.csv'
real_data_FP = (
    pd.read_csv(real_data_FP_url)
      .rename(columns={'Pos': 'POS', 'player_display_name': 'Player'})
      .replace({'-': 0, 'BYE': 0})
      .drop(columns = ['#', 'Team'], errors='ignore')
      .query("POS in ['K', 'DST']")
)

real_data_FP = real_data_FP.melt(
    id_vars=['Player', 'POS'],
    value_vars=[str(w) for w in range(1, 18)],
    var_name='week',
    value_name='fantasy_points_ppr'
)
real_data_FP = real_data_FP[~real_data_FP['fantasy_points_ppr'].isin(['BYE', '-', None])].copy()
real_data_FP['fantasy_points_ppr'] = pd.to_numeric(real_data_FP['fantasy_points_ppr'])
real_data = pd.concat([real_data_FP, real_data_RZ], ignore_index=True)

# print("Positions in Fantasy Pros Data:")
# print(real_data_FP['POS'].value_counts())
# print("\nPositions in Row Zero Data:")
# print(real_data_RZ['POS'].value_counts())
# print("\nPositions in Concatonated Data")
# print(real_data['POS'].value_counts())

# Real Projections laden
real_projections_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/real_projections/real_projections_{year}.csv'
real_projections = (
    pd.read_csv(real_projections_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'points': 'Projection', 'week': 'Week'})
      .loc[:, ['Player', 'POS', 'Projection', 'Week']]
      .pivot_table(
         index=['Player', 'POS'],
         columns='Week',
         values='Projection',
         aggfunc='first'
      )
      .rename_axis(None, axis=1)
      .add_prefix('Week_')
      .reset_index()
      .fillna(0)
      .drop_duplicates(subset='Player', keep='first')
)

# Spieler-Namen matchen (aus verschiedenen Datenquellen)
adp_names = player_adp['Player'].unique().tolist()
adp_pos_dict = dict(zip(player_adp['Player'], player_adp['POS']))

def match_name_with_pos(name, pos, reference_list, reference_pos_dict, cutoff=85.5):
    filtered_candidates = [p for p in reference_list if reference_pos_dict.get(p) == pos]
    result = process.extractOne(name, filtered_candidates, score_cutoff=cutoff)
    return result[0] if result else None

real_data['Player'] = real_data.apply(
    lambda row: match_name_with_pos(row['Player'], row['POS'], adp_names, adp_pos_dict),
    axis=1
)
real_data = real_data.drop_duplicates(subset=['Player', 'POS', 'week'], keep='first')

real_data = real_data[real_data['Player'].notnull()].copy()

# Block 2 - Funktionen definieren


In [ ]:
teams = [f'Team {i+1}' for i in range(num_teams)]
num_reg_weeks = 14
playoff_weeks = [15, 16, 17]

def create_reg_schedule(teams):
    n = len(teams)
    schedule = []
    for week in range(num_reg_weeks):
        week_matches = []
        for i in range(n//2):
            team1 = teams[i]
            team2 = teams[n-1-i]
            week_matches.append((team1, team2))
        schedule.append(week_matches)
        # Rotate teams except the first one
        teams = [teams[0]] + [teams[-1]] + teams[1:-1]
    return schedule

reg_schedule = create_reg_schedule(teams)

def simulate_league(seed, num_teams, num_rounds, player_adp, real_projections, real_data, reg_schedule):
    import random
    import pandas as pd

    random.seed(seed)

    teams = [f'Team {i+1}' for i in range(num_teams)]
    players_list = player_adp['Player'].copy().tolist()
    pos = dict(zip(player_adp['Player'], player_adp['POS']))
    df_sorted = player_adp.sort_values('Rank').reset_index(drop=True)

    draft_order = []
    for rnd in range(num_rounds):
        order = teams if rnd % 2 == 0 else teams[::-1]
        draft_order += order

    lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1, 'FLEX': 1}
    min_roster_req = lineup_req.copy()
    min_roster_req['RB'] += 1
    min_roster_req['WR'] += 1
    min_roster_req['TE'] += 1

    def opponent_pick(roster, available, Rk, min_req, picks_remaining, round_number):
        pos_counts = {p: sum(1 for pl in roster if pos[pl] == p) for p in min_req}
        needed = {p: max(0, min_req[p] - pos_counts.get(p, 0)) for p in min_req}
        total_needed = sum(needed.values())

        if total_needed >= picks_remaining:
            for p in sorted(needed, key=lambda k: needed[k], reverse=True):
                candidates = [pl for pl in available if pos[pl] == p]
                if candidates:
                    return min(candidates, key=lambda x: Rk[x])
        remaining_players = sorted(available, key=lambda p: Rk[p])
        topk = min(len(remaining_players), max(1, round_number * 2))
        return random.choice(remaining_players[:topk])

    rosters = {tm: [] for tm in teams}
    available = set(players_list)
    draft_log = []

    for pick_idx, team in enumerate(draft_order, start=1):
        rank_dict = dict(zip(df_sorted['Player'], df_sorted['Rank']))
        remaining_players = sorted(available, key=lambda p: rank_dict.get(p, float('inf')))
        Rk = {p: i + 1 for i, p in enumerate(remaining_players)}
        picks_remaining = num_rounds - len(rosters[team])
        current_round = (pick_idx - 1) // len(teams) + 1

        pick = opponent_pick(
            roster=rosters[team],
            available=available,
            Rk=Rk,
            min_req=min_roster_req,
            picks_remaining=picks_remaining,
            round_number=current_round
        )

        rosters[team].append(pick)
        available.remove(pick)
        draft_log.append({
            'Pick': pick_idx,
            'Team': team,
            'Player': pick,
            'Round': current_round,
            'POS': pos[pick],
            'ADP': player_adp.loc[player_adp['Player'] == pick, 'ADP'].values[0]
        })

    df_draft = pd.DataFrame(draft_log)
    roster_weekly_projections = (
        df_draft
        .merge(real_projections.drop(columns='POS'), on='Player', how='left')
        .sort_values(by=['Team', 'Pick'])
        .reset_index(drop=True)
    )

    def get_best_lineup(team, week, roster_weekly_projections, lineup_req):
        week_col = f'Week_{week}'
        team_roster = roster_weekly_projections[roster_weekly_projections['Team'] == team]
        lineup = []
        used_players = set()
        for pos, limit in lineup_req.items():
            if pos != 'FLEX':
                candidates = team_roster[team_roster['POS'] == pos]
            else:
                candidates = team_roster[
                    (team_roster['POS'].isin(['RB', 'WR', 'TE'])) &
                    (~team_roster['Player'].isin(used_players))
                ]
            starters = candidates.sort_values(week_col, ascending=False).head(limit)
            lineup.append(starters)
            used_players.update(starters['Player'])
        return pd.concat(lineup)

    def get_actual_points(lineup, week, real_data):
        merged = lineup.merge(
            real_data[real_data['week'] == week],
            left_on='Player', right_on='Player', how='left'
        )
        merged['fantasy_points_ppr'] = merged['fantasy_points_ppr'].fillna(0)
        return merged['fantasy_points_ppr'].sum()

    results = []
    for week_idx, matchups in enumerate(reg_schedule, 1):
        for team1, team2 in matchups:
            lineup1 = get_best_lineup(team1, week_idx, roster_weekly_projections, lineup_req)
            lineup2 = get_best_lineup(team2, week_idx, roster_weekly_projections, lineup_req)
            points1 = get_actual_points(lineup1, week_idx, real_data)
            points2 = get_actual_points(lineup2, week_idx, real_data)
            winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
            results.append({'Woche': week_idx, 'Sieger': winner})

    df_results = pd.DataFrame(results)
    wins = df_results[df_results['Sieger'] != 'Unentschieden']['Sieger'].value_counts()
    record = pd.DataFrame({'Team': teams})
    record['Wins'] = record['Team'].map(wins).fillna(0).astype(int)
    record = record.sort_values(by=['Wins'], ascending=[False]).reset_index(drop=True)
    ranked_teams = record['Team'].tolist()

    week15 = [(ranked_teams[2], ranked_teams[5]), (ranked_teams[3], ranked_teams[4])]
    winners15 = []
    for team1, team2 in week15:
        p1 = get_actual_points(get_best_lineup(team1, 15, roster_weekly_projections, lineup_req), 15, real_data)
        p2 = get_actual_points(get_best_lineup(team2, 15, roster_weekly_projections, lineup_req), 15, real_data)
        winner = team1 if p1 > p2 else team2
        winners15.append(winner)

    week16 = [(winners15[0], ranked_teams[1]), (winners15[1], ranked_teams[0])]
    winners16 = []
    for team1, team2 in week16:
        p1 = get_actual_points(get_best_lineup(team1, 16, roster_weekly_projections, lineup_req), 16, real_data)
        p2 = get_actual_points(get_best_lineup(team2, 16, roster_weekly_projections, lineup_req), 16, real_data)
        winner = team1 if p1 > p2 else team2
        winners16.append(winner)

    team1, team2 = winners16
    p1 = get_actual_points(get_best_lineup(team1, 17, roster_weekly_projections, lineup_req), 17, real_data)
    p2 = get_actual_points(get_best_lineup(team2, 17, roster_weekly_projections, lineup_req), 17, real_data)
    champion = team1 if p1 > p2 else team2

    points_for_summary = []
    for team in teams:
        points_reg_season = 0
        points_full_season = 0
        for week in range(1, 18):
            lineup = get_best_lineup(team, week, roster_weekly_projections, lineup_req)
            total_points = get_actual_points(lineup, week, real_data)
            if week <= 14:
                points_reg_season += total_points
            points_full_season += total_points
        points_for_summary.append({
            'Team': team,
            'Points Reg Season': round(points_reg_season, 2),
            'Points Full Season': round(points_full_season, 2)
        })

    df_points = pd.DataFrame(points_for_summary)
    df_champ_roster = df_draft[df_draft['Team'] == champion][['Player', 'ADP']]
    champion_roster = [f"{row.Player} / {row.ADP:.2f}" for row in df_champ_roster.itertuples()]
    points_row = df_points[df_points['Team'] == champion].iloc[0]

    return {
        'Winner': champion,
        'Roster': champion_roster,
        'Points Reg Season': points_row['Points Reg Season'],
        'Points Full Season': points_row['Points Full Season']
    }

In [ ]:
#For 2020 (16 Weeks)

num_reg_weeks = 13
playoff_weeks = [14, 15, 16]

def create_reg_schedule(teams):
    n = len(teams)
    schedule = []
    for week in range(num_reg_weeks):
        week_matches = []
        for i in range(n//2):
            team1 = teams[i]
            team2 = teams[n-1-i]
            week_matches.append((team1, team2))
        schedule.append(week_matches)
        # Rotate teams except the first one
        teams = [teams[0]] + [teams[-1]] + teams[1:-1]
    return schedule

reg_schedule = create_reg_schedule(teams)

def simulate_league(seed, num_teams, num_rounds, player_adp, real_projections, real_data, reg_schedule):
    import random
    import pandas as pd

    random.seed(seed)

    teams = [f'Team {i+1}' for i in range(num_teams)]
    players_list = player_adp['Player'].copy().tolist()
    pos = dict(zip(player_adp['Player'], player_adp['POS']))
    df_sorted = player_adp.sort_values('Rank').reset_index(drop=True)

    draft_order = []
    for rnd in range(num_rounds):
        order = teams if rnd % 2 == 0 else teams[::-1]
        draft_order += order

    lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1, 'FLEX': 1}
    min_roster_req = lineup_req.copy()
    min_roster_req['RB'] += 1
    min_roster_req['WR'] += 1
    min_roster_req['TE'] += 1

    def opponent_pick(roster, available, Rk, min_req, picks_remaining, round_number):
        pos_counts = {p: sum(1 for pl in roster if pos[pl] == p) for p in min_req}
        needed = {p: max(0, min_req[p] - pos_counts.get(p, 0)) for p in min_req}
        total_needed = sum(needed.values())

        if total_needed >= picks_remaining:
            for p in sorted(needed, key=lambda k: needed[k], reverse=True):
                candidates = [pl for pl in available if pos[pl] == p]
                if candidates:
                    return min(candidates, key=lambda x: Rk[x])
        remaining_players = sorted(available, key=lambda p: Rk[p])
        topk = min(len(remaining_players), max(1, round_number * 2))
        return random.choice(remaining_players[:topk])

    rosters = {tm: [] for tm in teams}
    available = set(players_list)
    draft_log = []

    for pick_idx, team in enumerate(draft_order, start=1):
        rank_dict = dict(zip(df_sorted['Player'], df_sorted['Rank']))
        remaining_players = sorted(available, key=lambda p: rank_dict.get(p, float('inf')))
        Rk = {p: i + 1 for i, p in enumerate(remaining_players)}
        picks_remaining = num_rounds - len(rosters[team])
        current_round = (pick_idx - 1) // len(teams) + 1

        pick = opponent_pick(
            roster=rosters[team],
            available=available,
            Rk=Rk,
            min_req=min_roster_req,
            picks_remaining=picks_remaining,
            round_number=current_round
        )

        rosters[team].append(pick)
        available.remove(pick)
        draft_log.append({
            'Pick': pick_idx,
            'Team': team,
            'Player': pick,
            'Round': current_round,
            'POS': pos[pick],
            'ADP': player_adp.loc[player_adp['Player'] == pick, 'ADP'].values[0]
        })

    df_draft = pd.DataFrame(draft_log)
    roster_weekly_projections = (
        df_draft
        .merge(real_projections.drop(columns='POS'), on='Player', how='left')
        .sort_values(by=['Team', 'Pick'])
        .reset_index(drop=True)
    )

    def get_best_lineup(team, week, roster_weekly_projections, lineup_req):
        week_col = f'Week_{week}'
        team_roster = roster_weekly_projections[roster_weekly_projections['Team'] == team]
        lineup = []
        used_players = set()
        for pos, limit in lineup_req.items():
            if pos != 'FLEX':
                candidates = team_roster[team_roster['POS'] == pos]
            else:
                candidates = team_roster[
                    (team_roster['POS'].isin(['RB', 'WR', 'TE'])) &
                    (~team_roster['Player'].isin(used_players))
                ]
            starters = candidates.sort_values(week_col, ascending=False).head(limit)
            lineup.append(starters)
            used_players.update(starters['Player'])
        return pd.concat(lineup)

    def get_actual_points(lineup, week, real_data):
        merged = lineup.merge(
            real_data[real_data['week'] == week],
            left_on='Player', right_on='Player', how='left'
        )
        merged['fantasy_points_ppr'] = merged['fantasy_points_ppr'].fillna(0)
        return merged['fantasy_points_ppr'].sum()

    results = []
    for week_idx, matchups in enumerate(reg_schedule, 1):
        for team1, team2 in matchups:
            lineup1 = get_best_lineup(team1, week_idx, roster_weekly_projections, lineup_req)
            lineup2 = get_best_lineup(team2, week_idx, roster_weekly_projections, lineup_req)
            points1 = get_actual_points(lineup1, week_idx, real_data)
            points2 = get_actual_points(lineup2, week_idx, real_data)
            winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
            results.append({'Woche': week_idx, 'Sieger': winner})

    df_results = pd.DataFrame(results)
    wins = df_results[df_results['Sieger'] != 'Unentschieden']['Sieger'].value_counts()
    record = pd.DataFrame({'Team': teams})
    record['Wins'] = record['Team'].map(wins).fillna(0).astype(int)
    record = record.sort_values(by=['Wins'], ascending=[False]).reset_index(drop=True)
    ranked_teams = record['Team'].tolist()

    week15 = [(ranked_teams[2], ranked_teams[5]), (ranked_teams[3], ranked_teams[4])]
    winners15 = []
    for team1, team2 in week15:
        p1 = get_actual_points(get_best_lineup(team1, 14, roster_weekly_projections, lineup_req), 14, real_data)
        p2 = get_actual_points(get_best_lineup(team2, 14, roster_weekly_projections, lineup_req), 14, real_data)
        winner = team1 if p1 > p2 else team2
        winners15.append(winner)

    week16 = [(winners15[0], ranked_teams[1]), (winners15[1], ranked_teams[0])]
    winners16 = []
    for team1, team2 in week16:
        p1 = get_actual_points(get_best_lineup(team1, 15, roster_weekly_projections, lineup_req), 15, real_data)
        p2 = get_actual_points(get_best_lineup(team2, 15, roster_weekly_projections, lineup_req), 15, real_data)
        winner = team1 if p1 > p2 else team2
        winners16.append(winner)

    team1, team2 = winners16
    p1 = get_actual_points(get_best_lineup(team1, 16, roster_weekly_projections, lineup_req), 16, real_data)
    p2 = get_actual_points(get_best_lineup(team2, 16, roster_weekly_projections, lineup_req), 16, real_data)
    champion = team1 if p1 > p2 else team2

    points_for_summary = []
    for team in teams:
        points_reg_season = 0
        points_full_season = 0
        for week in range(1, 17):
            lineup = get_best_lineup(team, week, roster_weekly_projections, lineup_req)
            total_points = get_actual_points(lineup, week, real_data)
            if week <= 14:
                points_reg_season += total_points
            points_full_season += total_points
        points_for_summary.append({
            'Team': team,
            'Points Reg Season': round(points_reg_season, 2),
            'Points Full Season': round(points_full_season, 2)
        })

    df_points = pd.DataFrame(points_for_summary)
    df_champ_roster = df_draft[df_draft['Team'] == champion][['Player', 'ADP']]
    champion_roster = [f"{row.Player} / {row.ADP:.2f}" for row in df_champ_roster.itertuples()]
    points_row = df_points[df_points['Team'] == champion].iloc[0]

    return {
        'Winner': champion,
        'Roster': champion_roster,
        'Points Reg Season': points_row['Points Reg Season'],
        'Points Full Season': points_row['Points Full Season']
    }



# Block 3 - Drafts simulieren

In [ ]:
num_batches = 10
num_simulations = 50

for batch_number in range(1, num_batches + 1):
    base_seed = (batch_number - 1) * num_simulations
    simulation_results = []
    champions = []

    for i in range(num_simulations):
        seed = base_seed + i + 1
        print(f"\nBatch {batch_number}, Simulation {i + 1} started...")
        start_time = time.time()

        sim_result = simulate_league(
            seed=seed,
            num_teams=num_teams,
            num_rounds=num_rounds,
            player_adp=player_adp,
            real_projections=real_projections,
            real_data=real_data,
            reg_schedule=reg_schedule
        )

        elapsed = round(time.time() - start_time, 2)

        simulation_results.append({
            'Simulation': i + 1,
            'Winner': sim_result['Winner'],
            'Roster': sim_result['Roster'],
            'Points Reg Season': sim_result['Points Reg Season'],
            'Points Full Season': sim_result['Points Full Season']
        })

        champions.append(sim_result['Winner'])
        print(f"  Done in {elapsed} seconds.")

    df_batch = pd.DataFrame(simulation_results)

    output_filename = f"simulation_results_ADP_{batch_number}_{year}.json"
    df_batch.to_json(output_filename, orient='records', indent=2)
    print(f"\nBatch {batch_number} gespeichert unter: {output_filename}")